In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
#计算多少个批次
n_batch = mnist.train.num_examples//batch_size

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_drop = tf.placeholder(tf.float32)
#构建神经网络
#防止过拟合加入dropout
W1 = tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))
b1 = tf.Variable(tf.zeros([2000])+0.1)
L1_pre = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1_pre,keep_drop)

W2 = tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.1)
L2_pre = tf.nn.tanh(tf.matmul(L1_pre,W2)+b2)
L2_drop = tf.nn.dropout(L2_pre,keep_drop)


W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
L3_pre = tf.nn.tanh(tf.matmul(L2_pre,W3)+b3)
L3_drop = tf.nn.dropout(L3_pre,keep_drop)


W4 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
pre = tf.nn.tanh(tf.matmul(L3_pre,W4)+b4)
#代价函数
# loss = tf.reduce_mean(tf.square(y-pre))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pre))#交叉熵
#使用梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_pre = tf.equal(tf.argmax(y,1),tf.argmax(pre,1))#argmax返回一维张量中最大值的位置 
                                                       #equal比较两个值得是否相同，相同返回Ture,反之false

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_pre,tf.float32))#cast函数将布尔型变成浮点型

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31):     #训练周期为31次
        for batch in range(n_batch):   #
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_drop:1.0})
        
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels})
        print("周期："+str(epoch)+","+"测试精度："+str(test_acc) +"," + "训练精度" + str(train_acc))

In [ ]:
#改动dropout只需要改keep_drop的值